In [0]:
%pip install mlflow==2.10.1 lxml==4.9.3 langchain==0.1.5 databricks-vectorsearch==0.22 cloudpickle==2.2.1 databricks-sdk==0.18.0 cloudpickle==2.2.1 pydantic==2.5.2 openpyxl
%pip install pip mlflow[databricks]==2.10.1

dbutils.library.restartPython()

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Window
import pandas as pd
import random
import os
import uuid
import json
import io
import plotly.express as px
from langchain_community.chat_models import ChatDatabricks

from datetime import datetime
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StringType

In [0]:
chat_endpoint = "databricks-llama-4-maverick" 
chat_model = ChatDatabricks(endpoint = chat_endpoint,
                            temperature = 0,
                            max_tokens = 8096) 

# Prompt Engineering    
**Definitions**    
  * **Prompt:** is an input or query given to a LLM to elict to a specific response or output        
  * **Prompt Engineering:** is the pratice of designing and refining prompts to optimize the responses generated by AI models.  

A good prompt usually consist of:    
  * Instruction    
  * Context    
  * input / question    
  * Output type / format

#### 1.  Zero-shot Prompting.   
Prompt for a generation of content, text, video, image etc... without providing any examples or additional training specific task.


##### Examples generated by ChatGPT:   
**Prompt one:**   
Translate the following sentence to French: “I love learning about artificial intelligence.”   
**LLM's answer one:**   
J'aime apprendre l'intelligence artificielle.   

**Prompt two:**   
Summarize this paragraph in one sentence: “Prompt engineering is the art of crafting inputs to guide large language models to perform useful tasks more effectively.”   
**LLM's answer two:**    
Prompt engineering is the practice of designing inputs to improve LLM performance.   

**Prompt three:**   
Classify the following tweet as positive, negative, or neutral: “The app keeps crashing after the update.”
**LLM's answer three:**    
Negative   


In [0]:
Prompt_one = "Translate the following sentence to French: “I love learning about artificial intelligence."
LLM_answer_one = chat_model.invoke(Prompt_one)

print(LLM_answer_one.content)

In [0]:
Prompt_two = "Summarize this paragraph in one sentence: “Prompt engineering is the art of crafting inputs to guide large language models to perform useful tasks more effectively."
LLM_answer_two = chat_model.invoke(Prompt_two)

print(LLM_answer_two.content)

In [0]:
Prompt_three = "Classify the following tweet as positive, negative, or neutral: “The app keeps crashing after the update."
LLM_answer_three = chat_model.invoke(Prompt_three)

print(LLM_answer_three.content)

#### 2. Few-shot Prompting.    
Prompt provides with a few input-ouput examples to guide the model for generating the desired outuput.

#### Example generated by ChatGPT    
**Prompt one:**    
Translate English to Spanish:   

Hello → Hola

Good night → Buenas noches   

Thank you → Gracias   
Translate: How are you?    
LLM's answer one:   
¿Cómo estás?   

**Prompt two:**    
Classify the sentiment:    
Review: “I love the design and speed of this phone.” → Positive    
Review: “The battery life is terrible.” → Negative   
Review: “It’s okay, not the best.” → Neutral    
Review: “Absolutely amazing performance!” →    
LLM's answer two:    
Positive    

**Prompt three:**    
Correct the grammar:    

“She go to school every day.” → “She goes to school every day.”    

“I has a dog.” → “I have a dog.”    
    
“He don’t like pizza.” →    
LLM's answer three:    
He doesn’t like pizza.    

In [0]:
prompt_one = """
Translate English to Spanish:

Hello → Hola

Good night → Buenas noches

Thank you → Gracias

Translate: How are you?"""

LLM_answer_one = chat_model.invoke(prompt_one)

print(LLM_answer_one.content)

In [0]:
prompt_two = """
Classify the sentiment:
Review: “I love the design and speed of this phone.” → Positive
Review: “The battery life is terrible.” → Negative
Review: “It’s okay, not the best.” → Neutral
Review: “Absolutely amazing performance!” →
"""
LLM_answer_two = chat_model.invoke(prompt_two)

print(LLM_answer_two.content)

In [0]:
prompt_three = """
Correct the grammar:

“She go to school every day.” → “She goes to school every day.”

“I has a dog.” → “I have a dog.”

“He don’t like pizza.” →
"""
LLM_answer_three = chat_model.invoke(prompt_three)
print(LLM_answer_three.content)

#### Prompting Chaining    
**Bread tasks into subtasks**    
Multi tasks are linked together, with the outupt of one prompt serving as the input of the next.   
This method allows for more complex tasks to be broken down into manageable steps

#### Examples generate by ChatGPT
🔹 Example 1: Article Summarization + Title Generation    
**Prompt one:**    
Summarize the following article in 3 sentences:    
"Prompt engineering is a technique used to guide large language models. It involves crafting input queries to get desired output behavior. As LLMs become more capable, prompt engineering is turning into a vital skill for developers and researchers."    
LLM's answer one:    
Prompt engineering is a method for shaping the responses of large language models through carefully designed inputs. It helps guide the model toward producing desired results. With the growth of LLM capabilities, this skill is becoming increasingly important.    

**Prompt two:**    
Using the summary below, generate a compelling article title:    
“Prompt engineering is a method for shaping the responses of large language models through carefully designed inputs. It helps guide the model toward producing desired results. With the growth of LLM capabilities, this skill is becoming increasingly important.”    
LLM's answer two:    
“The Rise of Prompt Engineering: Shaping the Future of AI Interaction”    

🔹 Example 2: Email to Bullet Points → Slide Content    
**Prompt one:**    
Extract 5 key bullet points from this email:    
"Hi team, the product launch is scheduled for August 10th. We need marketing assets ready by August 5th. Sales training will happen the same week. Please confirm if all departments are aligned. Also, QA must finish testing by August 4th."    
LLM's answer one:    

Product launch date: August 10th    

Marketing materials deadline: August 5th    

Sales training scheduled for launch week    

Request for departmental alignment    

QA testing deadline: August 4th    

**Prompt two:**    
Turn the following bullet points into concise slide content for a launch planning presentation:    

Product launch date: August 10th    

Marketing materials deadline: August 5th    

Sales training scheduled for launch week    

Request for departmental alignment    

QA testing deadline: August 4th    
LLM's answer two:    
Slide Title: Key Launch Deadlines    

Launch: August 10    

Marketing assets due: August 5    

QA complete by: August 4    

Sales training: Launch week    

Cross-team alignment required    

🔹 Example 3: Raw Idea → Blog Outline → Blog Introduction    
**Prompt one:**    
Turn this raw idea into a blog post outline:    
"How AI can support mental health and well-being in the workplace."    
LLM's answer one:    
Outline:    

Introduction: The rising focus on mental health    
    
Challenges in workplace well-being    

How AI tools can detect burnout or stress    

Virtual assistants for emotional support    

Ethical considerations and privacy    

Conclusion: The future of AI in mental wellness    

**Prompt two:**    
Write an engaging introduction paragraph based on this outline:    

Introduction: The rising focus on mental health        

Challenges in workplace well-being    

How AI tools can detect burnout or stress    

Virtual assistants for emotional support    

Ethical considerations and privacy    

Conclusion: The future of AI in mental wellness    
LLM's answer two:    
In recent years, mental health has become a central topic in conversations about workplace wellness. With rising stress levels and burnout rates, organizations are seeking innovative ways to support employee well-being. One promising avenue is the use of AI-driven tools that not only monitor emotional health but also provide real-time assistance. As this technology advances, it holds great potential to transform how we approach mental health at work.

In [0]:
# Example 1
prompt_one = """
Summarize the following article in 3 sentences:
'Prompt engineering is a technique used to guide large language models. It involves crafting input queries to get desired output behavior. As LLMs become more capable, prompt engineering is turning into a vital skill for developers and researchers.'"""

LLM_answer_one = chat_model.invoke(prompt_one)

print(LLM_answer_one.content)

prompt_two = f"""Using the summary below, generate a compelling article title:
{LLM_answer_one.content}"""


LLM_answer_two = chat_model.invoke(prompt_two)

print(LLM_answer_two.content)

In [0]:
# Example 2
prompt_one = """Extract 5 key bullet points from this email:
'Hi team, the product launch is scheduled for August 10th. We need marketing assets ready by August 5th. Sales training will happen the same week. Please confirm if all departments are aligned. Also, QA must finish testing by August 4th.'"""

LLM_answer_one = chat_model.invoke(prompt_one)

print(LLM_answer_one.content)

prompt_two = f"""Turn the following bullet points into concise slide content for a launch planning presentation:
{LLM_answer_one.content}"""


LLM_answer_two = chat_model.invoke(prompt_two)

print(LLM_answer_two.content)

In [0]:
# Example 3
prompt_one = """Turn this raw idea into a blog post outline:
'How AI can support mental health and well-being in the workplace.'"""

LLM_answer_one = chat_model.invoke(prompt_one)

print(LLM_answer_one.content)

prompt_two = f"""Write an engaging introduction paragraph based on this outline:
{LLM_answer_one.content}"""


LLM_answer_two = chat_model.invoke(prompt_two)

print(LLM_answer_two.content)

#### Chain-of-Thought Prompoting    
**Let's think step-by-step**   
CoT prompting enhaces the reasoning capabilities of LLMs by guiding them to articulare their thought process step-by-step, similar to human reasoning.  
Researsh in this area has mixed findings. So, the CoT prompt not always works as expected.

#### Examples generated by ChatGPT
**Prompt one:**   
If there are 12 cookies and 4 kids, how many cookies does each kid get?      
LLM's answer one:   
There are 12 cookies and 4 kids. Dividing 12 by 4 gives 3. So each kid gets 3 cookies.   

**Prompt two:**   
Tom is twice as old as Sarah. Sarah is 10 years old. How old is Tom?   
LLM's answer two:   
Sarah is 10. Tom is twice as old, so 10 × 2 = 20. Tom is 20 years old.   

**Prompt three:**   
John had 15 apples. He gave 3 to his friend and then bought 5 more. How many apples does he have now?   
LLM's answer three:   
John had 15 apples. He gave away 3, so now he has 12. Then he bought 5 more, so 12 + 5 = 17. He has 17 apples.   

In [0]:
prompt_one = """If there are 12 cookies and 4 kids, how many cookies does each kid get??
Let's thing step-by-step"""

LLM_answer_one = chat_model.invoke(prompt_one)

print(LLM_answer_one.content)

In [0]:
prompt_two = """Tom is twice as old as Sarah. Sarah is 10 years old. How old is Tom?Let's thing step-by-step"""
LLM_answer_two = chat_model.invoke(prompt_two)

print(LLM_answer_two.content)

In [0]:
prompt_three = """John had 15 apples. He gave 3 to his friend and then bought 5 more. How many apples does he have now?Let's thing step-by-step"""
LLM_answer_three = chat_model.invoke(prompt_three)
print(LLM_answer_three.content)

#### Instruction prompting (a.k.a. instruction tuning)
* Give explicit, clear instructions on what you want the model to do.

#### Example genereated by ChatGPT
**Prompt one:**    
Write a short paragraph (3–4 sentences) describing the importance of sleep for teenagers.    
LLM's answer one:    
Sleep is essential for teenagers because it supports growth and development. It also improves concentration and memory. Without enough rest, teens may struggle in school and experience mood swings. Adequate sleep contributes to both physical and mental health.    

**Prompt two:**    
List three benefits of using solar energy.    
LLM's answer two:    

It reduces electricity bills.    

It’s a clean and renewable energy source.    

It lowers carbon emissions.    

**Prompt three:**    
Explain in simple terms how a computer virus works.    
LLM's answer three:    
A computer virus is a program that spreads from one computer to another, just like a real virus. It can damage files, steal information, or make your computer act strangely. Viruses often hide inside other programs or files and activate when opened.

In [0]:
prompt_one = """Write a short paragraph (3–4 sentences) describing the importance of sleep for teenagers."""

LLM_answer_one = chat_model.invoke(prompt_one)

print(LLM_answer_one.content)

In [0]:
prompt_two = """List three benefits of using solar energy."""
LLM_answer_two = chat_model.invoke(prompt_two)

print(LLM_answer_two.content)

In [0]:
prompt_three = """Explain in simple terms how a computer virus works."""
LLM_answer_three = chat_model.invoke(prompt_three)
print(LLM_answer_three.content)

#### 5. Role prompting (a.k.a. persona prompting)
* Ask the model to play a specific role or persona before answering.

#### Examples generated by ChatGPT
P**rompt one:**    
You are a friendly science teacher. Explain gravity to a 10-year-old.    
LLM's answer one:    
Sure! Gravity is like an invisible force that pulls things toward the ground. It's the reason why we don’t float away and why things fall when you drop them. The Earth pulls everything toward it—just like a magnet!    

**Prompt two:**    
You are a professional fitness coach. Give three tips for staying active while working an office job.    
LLM's answer two:    

Take short standing or walking breaks every hour.    

Use a standing desk if possible.    

Do quick desk exercises like leg raises or stretches.    

**Prompt three:**    
You are a historian. Briefly explain the cause of World War I.    
LLM's answer three:    
World War I began due to a mix of political tensions, alliances, and militarism in Europe. The immediate cause was the assassination of Archduke Franz Ferdinand of Austria-Hungary in 1914. This event triggered a chain reaction of declarations of war between major powers.

In [0]:
prompt_one = """You are a friendly science teacher. Explain gravity to a 10-year-old."""

LLM_answer_one = chat_model.invoke(prompt_one)

print(LLM_answer_one.content)

In [0]:
prompt_two = """You are a professional fitness coach. Give three tips for staying active while working an office job."""
LLM_answer_two = chat_model.invoke(prompt_two)

print(LLM_answer_two.content)

In [0]:
prompt_three = """You are a historian. Briefly explain the cause of World War I."""
LLM_answer_three = chat_model.invoke(prompt_three)
print(LLM_answer_three.content)

# Tip: Formata prompts 
* Use dilimiters to distinguish between instruction and context   
  * Pound sign #####    
  * Backticks `````
  * Braces / brackets [] / {}    
  * Dashes -----
*Ask the model to return structured output
  * HTML, json, markdown, tec...   
* Provide a correct example
  * return your answer in the python dict format like {'some key':'some value'}

# Tip: Guide the model for better responses
* Ask the model not to make things up / Hallucinate
* Ask the model not to assume ro probe for sensitive information
* Ask the model not rush to a solution (think step-by-step)